**Calculate TD shot gather from FD modelling results**<br />
Daniel Köhn<br />
Kiel, 20.09.2017

**Import necessary packages**

In [ ]:
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
from matplotlib.pyplot import gca
import matplotlib as mpl
from pylab import rcParams
from matplotlib import rc
import matplotlib.ticker as mtick
from scipy import interpolate
from scipy import signal
#%matplotlib notebook

**Define parameters for data visualization**

In [ ]:
clip = 5e-2   # data clip
nsrc = 1     # number of shots
ntr = 48      # number of receivers
shotno = 1    # extract FDFD data for shot shotno

# number of frequencies nf and frequency interval df
FC_low = 0.5
FC_high = 100.0

nf = 400
df = (FC_high - FC_low) / (nf-1)
nfnsrc = nf * nsrc

# TD parameters
TmaxTD = 0.4          # maximum time TD
TmaxFD = 0.25 / df    # maximum time FD
dt = TmaxFD / nf      # time sampling FD

# maximum time sample of FD2TD corresponding to TmaxTD
nmaxFD = np.int(TmaxTD / dt)

# define time samples in time and frequency domain
tFD = np.arange(0.0, TmaxTD, dt)
offset = np.arange(0, ntr, 1)

fc = 40.0         # Ricker wavelet centre frequency [Hz]

**Font properties**

In [ ]:
FSize = 15
font = {'color':  'black',
        'weight': 'bold',
        'size': FSize}
mpl.rc('xtick', labelsize=FSize) 
mpl.rc('ytick', labelsize=FSize) 
rcParams['figure.figsize'] = 10, 10

__Time Damping__

In [ ]:
S = 0.0

**Read 2 cross TE-mode FD data**

In [ ]:
name_FDFD = "cachtice_p_stage_1.bin"
f = open (name_FDFD)
data_type = np.dtype ('float32').newbyteorder ('<')
data_FDFD = np.fromfile(f, dtype=data_type)
data_FDFD = data_FDFD.reshape(nf*nsrc,2*ntr)
data_FDFD = np.transpose(data_FDFD)
data_FDFD = np.flipud(data_FDFD)

**Extract FD data for shot shotno**

In [ ]:
FD_real = data_FDFD[0:2*ntr-1:2,shotno-1:nfnsrc:nsrc]
FD_imag = data_FDFD[1:2*ntr:2,shotno-1:nfnsrc:nsrc]
data_FDFD = None

# taper FD data
window = signal.tukey(nf,2)

for i in range (ntr):
    FD_real[i,:] *= window
    FD_imag[i,:] *= window

In [ ]:
# Define Ricker wavelet
t_stf = np.linspace(0, dt * nmaxFD, nmaxFD, endpoint=True)
ts = 1.0 / fc
tau = np.pi * (t_stf - 1.5 * ts) / (1.5 * ts)
wavelet = (1.0 - 4.0 * tau * tau) * np.exp(-2.0 * tau * tau) 

stf_real  = np.zeros((nf,1))
stf_imag  = np.zeros((nf,1))

f = FC_low
for i in range (nf):

    h=nmaxFD-1
    for j in range (nmaxFD):
            
        stf_real[i] += wavelet[j] * np.cos(2.0 * t_stf[j] * f * np.pi) * dt;
        stf_imag[i] += wavelet[j] * np.sin(2.0 * t_stf[j] * f * np.pi) * dt;
        h=h-1
        
    f +=df
    
FDFD_stf_real = np.zeros((nf, ntr))
FDFD_stf_imag = np.zeros((nf, ntr))
    
    
# copy STF FD data into 2D matrix
for i in range (nf):
    for j in range (ntr):
        
        FDFD_stf_real[i,j] = stf_real[i]
        FDFD_stf_imag[i,j] = stf_imag[i]

# assemble complex data
FDFD_stf = FDFD_stf_real + 1j*FDFD_stf_imag

In [ ]:
# assemble complex data
FDFD = FD_real + 1j*FD_imag
FD_real = None
FD_imag = None
FDFD = np.transpose(FDFD)

# convole spike data with source wavelet
tmp = FDFD * FDFD_stf
FDFD = None
FDFD = tmp

tmp1 = np.concatenate((FDFD, np.zeros((nf, ntr)), np.zeros((nf, ntr)), np.flipud(FDFD)))
#tmp1 = np.concatenate((np.zeros((1, ntr)),FDFD,np.flipud(FDFD)))
FDFD = None
FDFD = tmp1

In [ ]:
#plt.imshow(np.real(FDFD))
#plt.show()

**IFFT of FD data**

In [ ]:
# transformation from FD to TD
FD2TD = np.fft.ifft(FDFD,axis=0)
FD2TD = np.real(FD2TD)
tmp1 = None

**Extract FD2TD up to TmaxTD and resample FD2TD from dtFD to dt**

In [ ]:
# extract FD2TD data up to TmaxTD
tmp1 = FD2TD[1:nmaxFD,:]
FD2DTD = None

# clean memory
tmp = None

**Normalize data**

In [ ]:
#max_TD = np.max(traces)
max_FD = np.max(tmp1)
tmp1 = tmp1 / max_FD
FD2TD = np.flipud(tmp1)

tmp1 = None

__Load analytical solution__

In [ ]:
#TD_data = np.genfromtxt("../TE_analytical/analytical_hom.txt", dtype=None)
#TD_data /= np.amax(TD_data)
#TD_data = np.flipud(TD_data)

In [ ]:
#tmp1 = TD_data[:,0:29]
#tmp2 = TD_data[:,30:60]
#tmp3 = TD_data[:,61:88]
#tmp4 = TD_data[:,89:117]

#TD_data = None

#tmp = np.concatenate((np.flipud(tmp2.T), np.flipud(tmp3.T), tmp1.T, tmp4.T))
#TD_data = tmp.T

# calculate 1st temporal derivative 
#TD_data = np.diff(TD_data,axis=0)

#TD_data = TD_data / np.amax(TD_data)
#TD_data = np.fliplr(TD_data)

#tmp = None
#tmp1 = None
#tmp2 = None
#tmp3 = None
#tmp4 = None

**Plot shot gather**

In [ ]:
def do_plot(n, model, cm, an, title, clip):
    
    ax=plt.subplot(1,1,n)
    
    extent = [np.min(offset), np.max(offset), 0.0, dt * nmaxFD]

    plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
    plt.rc('text', usetex=True)

    im = plt.imshow(model, cmap=cm, interpolation='nearest', extent=extent, vmin=-clip, vmax=clip, aspect='auto')

    a = gca()
    a.set_xticklabels(a.get_xticks(), font)
    a.set_yticklabels(a.get_yticks(), font)
    
    #a.yaxis.set_major_formatter(mtick.FormatStrFormatter('%2.0d2'))
    a.xaxis.set_major_formatter(mtick.FormatStrFormatter('%.0d'))

    # Add the grid
    ax.grid(which='major', axis='both', linestyle='-')
    
    plt.title(title, fontdict=font)
    plt.ylabel('Time [s]', fontdict=font)
    #ax.set_yticklabels([])
    plt.xlabel('Trace no.', fontdict=font)
    plt.gca().invert_yaxis()

In [ ]:
plt.close('all')
plt.figure()
do_plot(1, -np.fliplr(FD2TD), 'gray', '', r"GERMAINE FDFD", clip)
#do_plot(2, TD_data, 'gray', '', r"Analytical Solution", clip)
plt.tight_layout()
output_file = "test_A0_NPML_20.png"
#plt.savefig(output_file, bbox_inches='tight', format='pdf')
plt.savefig(output_file, bbox_inches='tight', format='png')
plt.show()